## Homework 08: Classification

**Due:** Midnight on July 6th, BUT no late points will be charged if you get it in by the last late deadline. 

### Overview

In this final homework before starting our course project, we will introduce the essential machine learning paradigm of **classification**. We will work with the **UCI Adult** dataset. This is a binary classification task.

As we’ve discussed in this week’s lessons, the classification workflow is similar to what we’ve done for regression, with a few key differences:
- We use `StratifiedKFold` instead of plain `KFold` so that every fold keeps the original class proportions.
- We use classification metrics (e.g., accuracy, precision, recall, F1-score for binary classification) instead of regression metrics.
- We could explore misclassified instances through a confusion matrix (though we will not do that in this homework).

For this assignment, you’ll build a gradient boosting classification using `HistGradientBoostingClassifier` (HGBC) and explore ways of tuning the hyperparameters, including using the technique of early stopping, which basically avoiding have to tune the number of estimators (called `max_iter` in HGBC). 

HGBC has many advantages, which we explain below. 


### Grading

There are 7 graded problems, each worth 7 points, and you get 1 point free if you complete the assignment. 

In [7]:
pip install tqdm

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 18.3 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━ 4/6 [alembic]my]  WARNING: The script alembic is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━ 5/6 [optuna]  WARNING: The script optuna is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [optuna]
Note: you may need to restart the kernel to use updated packages.


In [17]:
# General utilities
import os
import io
import time
import zipfile
import requests
from collections import Counter

# Data handling and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import display
 
# Data source
from sklearn.datasets import fetch_openml

 
# scikit-learn core tools 
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold,
    RandomizedSearchCV
)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

 
# Import model 
from sklearn.ensemble import HistGradientBoostingClassifier
 
# Metrics
from sklearn.metrics import balanced_accuracy_score, classification_report
 
# Distributions for random search
from scipy.stats import loguniform, randint, uniform

# pandas dtypes helpers
from pandas.api.types import is_numeric_dtype, is_categorical_dtype
from pandas import CategoricalDtype

# Optuna Hyperparameter Search tool    (may need to be installed)
import optuna


# Misc

random_seed = 42

def format_hms(seconds):
    return time.strftime("%H:%M:%S", time.gmtime(seconds))



### Prelude 1: Load and Preprocess the UCI Adult Income Dataset

- Load the dataset from sklearn
- Preliminary EDA
- Feature Engineering 

In [18]:
# Load and clean
df = fetch_openml(name='adult', version=2, as_frame=True).frame

df.replace("?", np.nan, inplace=True)            # Some datasets use ? instead of Nan for missing data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             48842 non-null  int64   
 1   workclass       46043 non-null  category
 2   fnlwgt          48842 non-null  int64   
 3   education       48842 non-null  category
 4   education-num   48842 non-null  int64   
 5   marital-status  48842 non-null  category
 6   occupation      46033 non-null  category
 7   relationship    48842 non-null  category
 8   race            48842 non-null  category
 9   sex             48842 non-null  category
 10  capital-gain    48842 non-null  int64   
 11  capital-loss    48842 non-null  int64   
 12  hours-per-week  48842 non-null  int64   
 13  native-country  47985 non-null  category
 14  class           48842 non-null  category
dtypes: category(9), int64(6)
memory usage: 2.7 MB


#### Check: Is the dataset imbalanced?

In [19]:
print(df['class'].value_counts(normalize=True))

class
<=50K    0.760718
>50K     0.239282
Name: proportion, dtype: float64


**YES:** It looks like this dataset is somewhat imbalanced. Therefore, we will 
1. Tell the model to compensate during training by setting `class_weight='balanced'` when defining the model;
2. Evaluate it `balanced_accuracy` instead of `accuracy` and with class-aware metrics (precision, recall, F1); and
3. [Optional] Adjust the probability threshold instead of relying on raw accuracy alone after examining the precision-recall trade-off you observe at 0.5.
    

### Feature Engineering

Based on the considerations in **Appendix One**, we'll make the following changes to the dataset to facilitate training:


1. Drop `fnlwgt` and `education`.   
3. Replace `capital-gain` and `capital-loss` by their difference `capital_net` and add a log-scaled version `capital_net_log`.


In [20]:
# Drop the survey-weight column
df_eng = df.drop(columns=["fnlwgt"])

# Keep only the ordinal education feature
df_eng = df_eng.drop(columns=["education"])      # retain 'education-num'

# Combine capital gains and losses, add a log-scaled variant
df_eng["capital_net"]     = df_eng["capital-gain"] - df_eng["capital-loss"]
df_eng["capital_net_log"] = np.log1p(df_eng["capital_net"].clip(lower=0))
df_eng = df_eng.drop(columns=["capital-gain", "capital-loss"])

# check
df_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   age              48842 non-null  int64   
 1   workclass        46043 non-null  category
 2   education-num    48842 non-null  int64   
 3   marital-status   48842 non-null  category
 4   occupation       46033 non-null  category
 5   relationship     48842 non-null  category
 6   race             48842 non-null  category
 7   sex              48842 non-null  category
 8   hours-per-week   48842 non-null  int64   
 9   native-country   47985 non-null  category
 10  class            48842 non-null  category
 11  capital_net      48842 non-null  int64   
 12  capital_net_log  48842 non-null  float64 
dtypes: category(8), float64(1), int64(4)
memory usage: 2.2 MB


#### Separate target and split

Create the feature set `X` and the target set `y` (using `class` as the target) and split the dataset into 80% training and 20% testing sets, making sure to stratify.

In [21]:

X = df_eng.drop(columns=["class"])
y = (df_eng["class"] == ">50K").astype(int)

# Split (with stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=random_seed,
    stratify=y                           # So same proportion of classes in train and test sets
)

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape,  y_test.shape)

Train: (39073, 12) (39073,)
Test : (9769, 12) (9769,)


### Prelude 2: Create a data pipeline and the `HistGradientBoostingClassifier` model

Histogram-based gradient boosting improves on the standard version by:

* **Histogram splits:** bins each feature into ≤ `max_bins` quantiles (i.e., each bin is approximately the same size) and tests splits only between bins, slashing compute time and scaling to large data sets. Default for `max_bins` = 255. 
* **Native NaN handling:** treats missing values as their own bin—no imputation needed.
* **Native Categorical Support**: accepts integer-encoded categories directly and tests “category c vs. all others” splits, eliminating one-hot blow-ups and fake orderings.
* **Built-in early stopping:** stops training after no improvement in validation loss after `n_iter_no_change` rounds. `tol` defines "improvement" (default is 1e-7). 
* **Leaf shrinkage:** adds `l2_regularization`, which ridge-shrinks each leaf value (without changing tree shape) so tiny, noisy leaves have less effect.

>**Summary:**  Histogram-based GB trades a tiny approximation error (binning) for a **huge speed-up** and adds extra conveniences, making it the preferred choice for large tabular data sets. Tuning workflow relies on **Early stopping** to stop training before overfitting occurs. 

In [22]:
# Define a baseline model 

HGBC_model = HistGradientBoostingClassifier(
    # tree structure and learning rate
    learning_rate=0.1,            # These 5 parameters are at defaults for our baseline training in Problem 1             
    max_leaf_nodes=31,            # but will be tuned by randomized search in Problem 2 and Optuna in Problem 3               
    max_depth=None,               
    min_samples_leaf=20,          
    l2_regularization=0.0,        

    # bins and iteration
    max_bins=255,                 # default
    max_iter=500,                 # high enough for early stopping
    early_stopping=True,
    n_iter_no_change=20,
    validation_fraction=0.2,      # 20% monitored for early stopping
    tol=1e-7,                     # default tolerance for validation improvement

    # class imbalance
    class_weight="balanced",

    random_state=random_seed,
    verbose=0
)


### Create a pipeline appropriate for HGBC 

**Why use a `Pipeline` instead of encoding in the dataset first?**

* **Avoid data leakage.** In each CV fold, the `OrdinalEncoder` is refit only on that fold’s training data, so the validation split never influences the encoder.
* **Single, reusable object.** The pipeline bundles preprocessing + model, letting you call `fit`/`predict` on raw data anywhere (CV, Optuna, production) with identical behavior.
* **Compatible with search tools.** `cross_validate`, `GridSearchCV`, and Optuna expect an estimator that can be cloned and refit; a pipeline meets that requirement automatically.

Put simply, the pipeline gives you leak-free evaluation and portable, hassle-free tuning without extra code.


In [23]:
enc = OrdinalEncoder(
    handle_unknown="use_encoded_value",   # Allow unseen categories during transform
    unknown_value=-1,                     # Code for unseen categories
    encoded_missing_value=-2,             # Code for missing values (NaN)
    dtype=np.int64                        # Needed for HistGradientBoostingClassifier
)

# Categorical features
cat_cols = X.select_dtypes(exclude=["number"]).columns.tolist()

# Numeric features (everything that isn’t object / category)
num_cols = X.select_dtypes(include=["number"]).columns.tolist()

preprocess = ColumnTransformer(
    [("cat", enc, cat_cols),
     ("num", "passthrough", num_cols)]
)

pipelined_model = Pipeline([
    ("prep", preprocess),
    ("gb",   HGBC_model)
])

## Problem 1: Baseline Cross-Validation with F1

In this problem, you will run a baseline cross-validation evaluation of your `HistGradientBoostingClassifier` pipeline, using `HGBC_model` defined above. 

**Background:**

* Since the Adult dataset is imbalanced (about 24% positives, 76% negatives), accuracy alone is not reliable.
* We will use the **F1 score** as the evaluation metric, since it balances precision (avoiding false positives) and recall (avoiding false negatives) in a single measure. This is a fairer metric for imbalanced classification, where both types of error matter.
* We will apply **5-fold stratified cross-validation** to make sure each fold has the same proportion of the classes as the original dataset.
* Repeated cross-validation is optional and not required here, because the Adult dataset is large and `HistGradientBoostingClassifier` is robust to small sampling differences. 

**Instructions:**

1. Set up a `StratifiedKFold` cross-validation object with 5 splits, shuffling enabled, and `random_state=random_seed`.
2. Use `cross_val_score` to estimate the mean F1 score and its standard deviation across the folds.
3. Print out the mean and standard deviation of the F1 score, rounded to 4 decimal places.
4. Answer the graded question.


In [26]:
# Your code here

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
f1_scores = cross_val_score(pipelined_model, X, y, cv=cv, scoring="f1")

p1_mean = np.mean(f1_scores)


### Problem 1 Graded Answer

Set `a1` to the mean F1 score of the baseline model. 

In [27]:
 # Your answer here

a1 = p1_mean                     # replace 0 with an expression

In [28]:
# DO NOT change this cell in any way

print(f'a1 = {a1:.4f}')

a1 = 0.7123


## Problem 2: Hyperparameter Optimization with Randomized Search for F1

In this problem, you will tune your `pipelined_model` using `RandomizedSearchCV` to identify the best combination of tree structure and learning rate parameters that maximize the **F1 score**.

**Background:**
The F1 score is our main metric because it balances precision and recall on an imbalanced dataset. Optimizing hyperparameters for F1 ensures we manage both false positives and false negatives in a single measure.

**Instructions:**

1. Set up a randomized search over the following hyperparameter ranges, using appropriate random-number distributions:

   * `learning_rate` (log-uniform between 1e-3 and 0.3)
   * `max_leaf_nodes` (integer from 16 to 256)
   * `max_depth` (integer from 2 to 10)
   * `min_samples_leaf` (integer from 10 to 200)
   * `l2_regularization` (uniform between 0.0 and 2.0)
2. Use **5-fold stratified cross-validation**, with the same settings as in Problem 1.
3. Set `n_iter` to at least 100 trials. More trials will generally yield better results, if your time and machine allow.
4. After running the search, show a neatly formatted table of the top 5 results, using `display(...)` showing their mean F1 scores, standard deviation, and the chosen hyperparameter values.
5. Answer the graded question.




In [29]:
# Your code here

param_distributions = {
    "gb__learning_rate": loguniform(1e-3, 0.3),
    "gb__max_leaf_nodes": randint(16, 257),
    "gb__max_depth": randint(2, 11),
    "gb__min_samples_leaf": randint(10, 201),
    "gb__l2_regularization": uniform(0.0, 2.0)
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)

search = RandomizedSearchCV(
    estimator=pipelined_model,
    param_distributions=param_distributions,
    n_iter=100,
    scoring="f1",
    cv=cv,
    random_state=random_seed,
    verbose=1,
    n_jobs=-1,
    return_train_score=False
)

search.fit(X, y)
results_df = pd.DataFrame(search.cv_results_)
top5 = results_df.sort_values(by="mean_test_score", ascending=False).head(5)

columns_to_display = [
    "mean_test_score", "std_test_score",
    "param_gb__learning_rate", "param_gb__max_leaf_nodes",
    "param_gb__max_depth", "param_gb__min_samples_leaf",
    "param_gb__l2_regularization"
]
display(top5[columns_to_display].round(4))


Fitting 5 folds for each of 100 candidates, totalling 500 fits


,mean_test_score,std_test_score,param_gb__learning_rate,param_gb__max_leaf_nodes,param_gb__max_depth,param_gb__min_samples_leaf,param_gb__l2_regularization
59,0.7140,0.0049,0.2295,61,2,190,1.4954
55,0.7139,0.0047,0.1756,36,3,39,0.9735
19,0.7132,0.0050,0.1275,48,3,57,1.8299
99,0.7129,0.0051,0.2263,126,4,153,1.9001
98,0.7125,0.0036,0.1749,214,10,37,0.0217


np.float64(0.7139843373951789)

### Problem 2 Graded Answer

Set `a2` to the mean F1 score of the best model found. 

In [39]:
 # Your answer here

a2 = search.best_score_                     # replace 0 with your answer, may copy from the displayed results

In [40]:
# DO NOT change this cell in any way

print(f'a2 = {a2:.4f}')

a2 = 0.7140


## Problem 3: Hyperparameter Optimization with Optuna for F1

In this problem, you will explore **Optuna**, a powerful hyperparameter optimization framework, to identify the best combination of hyperparameters that maximize the F1 score of your `pipelined_model`.

**Background:**
Optuna uses a smarter sampling strategy than grid search or randomized search, allowing you to explore the hyperparameter space more efficiently. It also supports *pruning*, which can stop unpromising trials early to save time. This makes it a popular SOTA optimization tool.

**Before you start** browse the [Optuna documentation](https://optuna.org) and view the [tutorial video](https://optuna.readthedocs.io/en/stable/tutorial/index.html). 

As before, we focus on the **F1 score** because it balances precision and recall, making it more robust on an imbalanced dataset.

**Instructions:**

1. Define an Optuna objective function to optimize F1 score, sampling the exact same hyperparameter ranges you did in Problem 2 and using the same CV settings.  
3. Set up an Optuna study with a reasonable number of trials (e.g., 100–500 depending on runtime resources--on my machine Optuna runs about 10x faster than randomized search for the same number of trials, but YMMV).
4. After running the optimization, `display` a clean table with the top 5 trials showing their F1 scores and corresponding hyperparameter settings.
5. Answer the graded question. 

**Note:**  There are many resources on Optuna you can find on the web, but for this problem, you have my permission to let ChatGPT write the code for you. 

In [33]:
# Your code here

def objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 16, 256),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 10, 200),
        "l2_regularization": trial.suggest_float("l2_regularization", 0.0, 2.0)
    }

    model = HistGradientBoostingClassifier(
        learning_rate=params["learning_rate"],
        max_leaf_nodes=params["max_leaf_nodes"],
        max_depth=params["max_depth"],
        min_samples_leaf=params["min_samples_leaf"],
        l2_regularization=params["l2_regularization"],
        max_bins=255,
        max_iter=500,
        early_stopping=True,
        n_iter_no_change=20,
        validation_fraction=0.2,
        tol=1e-7,
        class_weight="balanced",
        random_state=random_seed,
        verbose=0
    )

    pipeline = Pipeline([
        ("prep", preprocess),
        ("gb", model)
    ])

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
    scores = cross_val_score(pipeline, X, y, cv=cv, scoring="f1")

    return np.mean(scores)

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=random_seed))
study.optimize(objective, n_trials=200, show_progress_bar=True)

top5 = sorted(study.trials, key=lambda t: t.value, reverse=True)[:5]
top5_df = pd.DataFrame([{
    "F1 Score": t.value,
    "learning_rate": t.params["learning_rate"],
    "max_leaf_nodes": t.params["max_leaf_nodes"],
    "max_depth": t.params["max_depth"],
    "min_samples_leaf": t.params["min_samples_leaf"],
    "l2_regularization": t.params["l2_regularization"]
} for t in top5])

display(top5_df.round(4))


[I 2025-07-01 23:02:14,266] A new study created in memory with name: no-name-709a17df-73e6-4842-bf77-fa8acc1dbe88
Best trial: 0. Best value: 0.69529:   0%|          | 1/200 [00:20<1:08:10, 20.56s/it]

[I 2025-07-01 23:02:34,824] Trial 0 finished with value: 0.6952899580869166 and parameters: {'learning_rate': 0.008468008575248327, 'max_leaf_nodes': 245, 'max_depth': 8, 'min_samples_leaf': 124, 'l2_regularization': 0.31203728088487304}. Best is trial 0 with value: 0.6952899580869166.


Best trial: 0. Best value: 0.69529:   1%|          | 2/200 [00:38<1:03:11, 19.15s/it]

[I 2025-07-01 23:02:52,986] Trial 1 finished with value: 0.6746784536793463 and parameters: {'learning_rate': 0.0024345423962016913, 'max_leaf_nodes': 29, 'max_depth': 9, 'min_samples_leaf': 124, 'l2_regularization': 1.416145155592091}. Best is trial 0 with value: 0.6952899580869166.


Best trial: 0. Best value: 0.69529:   2%|▏         | 3/200 [01:07<1:17:57, 23.74s/it]

...

Best trial: 83. Best value: 0.715096: 100%|█████████▉| 199/200 [23:14<00:06,  6.21s/it]

[I 2025-07-01 23:25:29,108] Trial 198 finished with value: 0.7137878697082003 and parameters: {'learning_rate': 0.29125824306515063, 'max_leaf_nodes': 225, 'max_depth': 2, 'min_samples_leaf': 19, 'l2_regularization': 0.15709345342339706}. Best is trial 83 with value: 0.7150961500692044.


Best trial: 83. Best value: 0.715096: 100%|██████████| 200/200 [23:19<00:00,  7.00s/it]

[I 2025-07-01 23:25:34,210] Trial 199 finished with value: 0.7138021786527855 and parameters: {'learning_rate': 0.2999169143043232, 'max_leaf_nodes': 234, 'max_depth': 2, 'min_samples_leaf': 14, 'l2_regularization': 0.3266899842526562}. Best is trial 83 with value: 0.7150961500692044.


,F1 Score,learning_rate,max_leaf_nodes,max_depth,min_samples_leaf,l2_regularization
0,0.7151,0.1400,160,4,16,0.3132
1,0.7151,0.2356,186,4,10,0.8610
2,0.7150,0.2989,246,2,17,0.1951
3,0.7148,0.2125,221,2,15,0.4014
4,0.7148,0.2314,231,2,12,0.2671


np.float64(0.7150961500692044)

### Problem 3 Graded Answer

Set `a3` to the mean F1 score of the best model found. 

In [37]:
 # Your answer here

a3 = top5_df["F1 Score"][0]                     # replace 0 with your answer, may copy from the displayed results

In [38]:
# DO NOT change this cell in any way

print(f'a3 = {a3:.4f}')

a3 = 0.7151


## Problem 4: Final Model Evaluation on Test Set

In this problem, you will take the best hyperparameter configuration you found in your earlier experiments (Randomized Search or Optuna) and fully evaluate the resulting model on the test set.

**Background:**
When performing hyperparameter tuning, we typically optimize for a single metric (e.g., F1). However, before deployment, it is essential to check **all relevant metrics** on the final test set to understand the model’s behavior in a balanced way.

**Instructions:**

1. Take the best hyperparameters you found in Problems 2 or 3 and apply them to your `pipelined_model`.
2. Re-train this final tuned model on the **entire training set** (not just the folds).
3. Evaluate the final model on the heldout **test set**, reporting the following metrics:

   * Precision
   * Recall
   * F1 score
   * Balanced accuracy
4. Use `classification_report` **on the test set** to print precision, recall, and F1 score, and use `balanced_accuracy_score` separately to calculate and print balanced accuracy.
5. Answer the graded questions.

**Note:** We evaluate the metrics on the test set because it was never seen during training or hyperparameter tuning. This gives us an unbiased estimate of how the model will perform on truly unseen data. Evaluating on the training set would be misleading, because the model has already learned from that data and could appear artificially good.


In [42]:
best_params = search.best_params_
best_params

{'gb__l2_regularization': np.float64(1.4954375477948278),
 'gb__learning_rate': np.float64(0.2295485371853974),
 'gb__max_depth': 2,
 'gb__max_leaf_nodes': 61,
 'gb__min_samples_leaf': 190}

In [58]:
# Your code here

best_params = search.best_params_

# Step 1: Construct the final model using best hyperparameters
final_model = HistGradientBoostingClassifier(
    learning_rate=best_params["gb__learning_rate"],
    max_leaf_nodes=best_params["gb__max_leaf_nodes"],
    max_depth=best_params["gb__max_depth"],
    min_samples_leaf=best_params["gb__min_samples_leaf"],
    l2_regularization=best_params["gb__l2_regularization"],
    max_bins=255,
    max_iter=500,
    early_stopping=True,
    n_iter_no_change=20,
    validation_fraction=0.2,
    tol=1e-7,
    class_weight="balanced",
    random_state=random_seed,
    verbose=0
)

# Step 2: Rebuild full pipeline with final model
final_pipeline = Pipeline([
    ("prep", preprocess),
    ("gb", final_model)
])

# Step 3: Retrain on full training set
final_pipeline.fit(X_train, y_train)

# Step 4: Evaluate on the test set
y_pred = final_pipeline.predict(X_test)

# Report classification metrics
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

classif = classification_report(y_test, y_pred, digits=4, output_dict=True)

# Report balanced accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {bal_acc:.4f}")


Classification Report:
              precision    recall  f1-score   support

           0     0.9523    0.8217    0.8822      7431
           1     0.6053    0.8691    0.7136      2338

    accuracy                         0.8330      9769
   macro avg     0.7788    0.8454    0.7979      9769
weighted avg     0.8692    0.8330    0.8418      9769

Balanced Accuracy: 0.8454


### Problem 4 Graded Questions

- Set `a4a` to the balanced accuracy score of the best model.
- Set `a4b` to the macro average precision of this model.
- Set `a4c` to the macro average recall score of the this model.

**Note:** Macro average takes the mean of each class’s precision/recall without considering how many samples each class has, which is appropriate for a balanced evaluation.

In [52]:
 # Your answer here

a4a = bal_acc                     # replace 0 with your answer, use variable or expression from above

In [53]:
# DO NOT change this cell in any way

print(f'a4a = {a4a:.4f}')

a4a = 0.8454


In [54]:
 # Your answer here

a4b = classif["macro avg"]["precision"]                     # replace 0 with your answer, may copy from the displayed results

In [55]:
# DO NOT change this cell in any way

print(f'a4b = {a4b:.4f}')

a4b = 0.7788


In [56]:
 # Your answer here

a4c = classif["macro avg"]["recall"]                     # replace 0 with your answer, may copy from the displayed results

In [57]:
# DO NOT change this cell in any way

print(f'a4c = {a4c:.4f}')

a4c = 0.8454


## Problem 5: Understanding Precision, Recall, F1, and Balanced Accuracy

**Tutorial**

In binary classification, you will often evaluate these key metrics:

* **Precision**: *Of all the positive predictions the model made, how many were actually correct?*

  * High precision = few false positives
  * Low precision = many false positives

* **Recall**: *Of all the actual positive cases, how many did the model correctly identify?*

  * High recall = few false negatives
  * Low recall = many false negatives

* **F1 score**: The harmonic mean of precision and recall, which balances them in a single measure.

  * F1 is **highest** when precision and recall are both high and similar in value.
  * If precision and recall are unbalanced, F1 will drop to reflect that imbalance.

* **Balanced accuracy**: The average of recall across both classes (positive and negative).

  * It ensures the classifier is performing reasonably well on *both* groups, correcting for class imbalance.
  * Balanced accuracy is especially important if the classes are very unequal in size.

**Typical trade-offs to remember:**

* **Higher recall, lower precision**: the model finds most true positives but also mislabels some negatives as positives
* **Higher precision, lower recall**: the model is strict about positive predictions, but misses some true positives
* **Balanced precision and recall (good F1)**: a practical compromise
* **Balanced accuracy**: checks fairness across both classes

###  Problem 5 Graded Question (multiple choice)

A bank uses your model to identify customers earning over $50K for a premium product invitation. Based on your final test set evaluation, including macro-averaged precision and recall, which of the following best describes what might happen?

(1) The bank will miss some eligible high-income customers, but will avoid marketing mistakes by sending invitations only to those it is  confident about.

(2) The bank will successfully reach most high-income customers, but will also waste resources sending invitations to some low-income customers.

(3) The bank will perfectly identify all high-income and low-income customers, resulting in no wasted invitations and no missed opportunities.


In [59]:
 # Your answer here

a5 = 2                     # replace 0 with one of 1, 2, or 3

In [60]:
# DO NOT change this cell in any way

print(f'a5 = {a5}')

a5 = 2


### Appendix One: Feature Engineering

Here are some practical feature-engineering tweaks worth considering (beyond simply ordinal-encoding the categoricals)

| Feature(s)                                                           | Why the tweak can help                                                                                                                                                     | How to do it (quick version)                                                                                                                                                    | Keep / drop?      |
| -------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------- |
| **`fnlwgt`**                                                         | Survey sampling weight, not a predictor. Leaving it in often lets the model “cheat.”                                                                                       | `df = df.drop(columns=["fnlwgt"])`                                                                                                                                              | **Drop**          |
| **`education` *vs.* `education-num`**                                | They encode the **same** information twice (categorical label and its ordinal rank). Keeping both is redundant and can cause leakage of a perfectly predictive feature.    | Usually keep **only one**. For tree models `education-num` is simplest: `df = df.drop(columns=["education"])`                                                                   | **Drop one**      |
| **`capital-gain`, `capital-loss`**                                   | Highly skewed; most values are zero with a long upper tail. The sign (gain vs. loss) matters, but treating them separately wastes a feature slot.                          | 1) Combine: `df["capital_net"] = df["capital-gain"] - df["capital-loss"]`; 2) Log-transform to reduce skew: `df["capital_net_log"] = np.log1p(df["capital_net"].clip(lower=0))` | Replace originals |
| **`age`, `hours-per-week`**                                          | Continuous but with natural plateaus—trees handle splits fine, yet log or square-root scaling can soften extreme values; bucketing makes partial-dependence plots clearer. | Simple bucket: `df["age_bin"] = pd.cut(df["age"], bins=[16,25,35,45,55,65,90])` (optional)                                                                                      | Optional          |
| **Missing categories** (`workclass`, `occupation`, `native-country`) | HGB handles `-1`/`-2` codes fine, but you may want *explicit* “Missing” bucket for interpretability.                                                                       | Use `encoded_missing_value=-2` during encoding.                                                                                                            | Keep as is        |
| **Rare categories in `native-country`**                              | Hundreds of low-frequency countries dilute signal; grouping boosts stability.                                                                                              | Map infrequent categories to “Other”:                                                                                                                                           |                   |


#### Minimum set of tweaks (good baseline, low effort)

1. **Drop `fnlwgt`.**  
2. **Keep `education-num`, drop `education`.**  
3. **Combine `capital-gain` and `capital-loss` into `capital_net`** (optionally add a log-scaled version).  
4. Leave other numeric/categorical features as is; your histogram-GBDT will cope.


